In [1]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

In [2]:
conn = sqlite3.connect('chuong_3_trang.db')
cursor =  conn.cursor()

In [3]:
cursor.execute('''
            CREATE TABLE DATA(
                ID integer primary key,
                Num1 float,
                Num2 float)
''')

cursor.execute(''' 
            insert into DATA values
                (1,3.5,7.2),
                (2, 5.1, 8.4),
                (3, 7.8, 6.9),
                (4, 9.0, 10.2),
                (5, 11.3, 4.5),
                (6, 13.7, 12.8)
               ''')
conn.commit()

In [4]:
cursor.execute(''' 
               select * from DATA
               ''')
result  = cursor.fetchall()
print(result)

[(1, 3.5, 7.2), (2, 5.1, 8.4), (3, 7.8, 6.9), (4, 9.0, 10.2), (5, 11.3, 4.5), (6, 13.7, 12.8)]


bai1

In [5]:

cursor.execute('''
        WITH metric AS (
            select
               count(*) AS n,
               sum(Num1*Num2) AS sum_ab,
               sum(Num1) AS sum_a,
               sum(Num2) AS sum_b,
               sum(Num1 * Num1) AS sum_a2,
               sum(Num2 * Num2) AS sum_b2
        from DATA)
        select 
               (n * sum_ab-sum_a*sum_b)/
               (sqrt(n*sum_a2- sum_a*sum_a)*sqrt(n*sum_b2-sum_b*sum_b)) AS R_AB
               from metric
 ''')
result  = cursor.fetchall()
for i in result:
    print(f'Sự tương quan giữa a(num1) và b(num2) : R(a,b)={i[0]}')

Sự tương quan giữa a(num1) và b(num2) : R(a,b)=0.36331684731706154


bai 2

In [6]:
# Tạo bảng để lưu dữ liệu
cursor.execute('''
CREATE TABLE IF NOT EXISTS car_scores (
    day_id integer,
    model TEXT,
    score float
)
''')

In [7]:
data = [
    (1, 'A', 8), (1, 'B', 9), (1, 'C', 7),
    (2, 'A', 7.5), (2, 'B', 8.5), (2, 'C', 7),
    (3, 'A', 6), (3, 'B', 7), (3, 'C', 8),
    (4, 'A', 7), (4, 'B', 6), (4, 'C', 5)
]

# Thêm dữ liệu vào bảng
cursor.executemany("INSERT INTO car_scores VALUES (?, ?, ?)", data)
conn.commit()

In [8]:
cursor.execute("SELECT * FROM car_scores")
print("Dữ liệu trong cơ sở dữ liệu:")
for row in cursor.fetchall():
    print(row)

Dữ liệu trong cơ sở dữ liệu:
(1, 'A', 8.0)
(1, 'B', 9.0)
(1, 'C', 7.0)
(2, 'A', 7.5)
(2, 'B', 8.5)
(2, 'C', 7.0)
(3, 'A', 6.0)
(3, 'B', 7.0)
(3, 'C', 8.0)
(4, 'A', 7.0)
(4, 'B', 6.0)
(4, 'C', 5.0)


In [9]:
cursor.execute("""
SELECT model, day_id, score
FROM car_scores
ORDER BY model, day_id
""")

rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=['model', 'day_id', 'score'])

# Chuyển dữ liệu về định dạng bảng 
pivot_table = df.pivot_table(index='day_id', columns='model', values='score')
print("\nBảng dữ liệu:")
print(pivot_table)


Bảng dữ liệu:
model     A    B    C
day_id               
1       8.0  9.0  7.0
2       7.5  8.5  7.0
3       6.0  7.0  8.0
4       7.0  6.0  5.0


In [10]:
def categorize_score(score):
    if score <= 6:
        return 'Thấp'
    elif score <= 8:
        return 'Trung bình'
    else:
        return 'Cao'

df['score_category'] = df['score'].apply(categorize_score)

# Tạo bảng liên hệ 
contingency_table = pd.crosstab(df['day_id'], df['model'], 
                                values=df['score_category'], 
                                aggfunc=lambda x: ''.join(x))

print("\nBảng phân loại điểm theo ngày và mẫu xe:")
print(contingency_table)

observed = pd.crosstab(df['day_id'], [df['model'], df['score_category']])
print("\nBảng tần số quan sát:")
print(observed)


Bảng phân loại điểm theo ngày và mẫu xe:
model            A           B           C
day_id                                    
1       Trung bình         Cao  Trung bình
2       Trung bình         Cao  Trung bình
3             Thấp  Trung bình  Trung bình
4       Trung bình        Thấp        Thấp

Bảng tần số quan sát:
model             A              B                    C           
score_category Thấp Trung bình Cao Thấp Trung bình Thấp Trung bình
day_id                                                            
1                 0          1   1    0          0    0          1
2                 0          1   1    0          0    0          1
3                 1          0   0    0          1    0          1
4                 0          1   0    1          0    1          0


In [11]:
contingency = pd.crosstab(df['day_id'], df['model'], values=df['score'], aggfunc='mean')
chi2, p, dof, expected = chi2_contingency(contingency)

print("\nKết quả kiểm định chi-square:")
print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p}")
print("\nBảng tần số kỳ vọng:")
print(pd.DataFrame(expected, index=contingency.index, columns=contingency.columns))

alpha = 0.05
if p < alpha:
    print(f"\n==>  Có sự khác biệt đáng kể giữa các mẫu xe dựa trên điểm số (p-value = {p:.4f} < {alpha})")
else:
    print(f"\n==> Không có sự khác biệt đáng kể giữa các mẫu xe dựa trên điểm số (p-value = {p:.4f} > {alpha})")



Kết quả kiểm định chi-square:
Chi-square statistic: 0.826439691310499
p-value: 0.9913459315666546

Bảng tần số kỳ vọng:
model          A         B         C
day_id                              
1       7.953488  8.511628  7.534884
2       7.622093  8.156977  7.220930
3       6.959302  7.447674  6.593023
4       5.965116  6.383721  5.651163

==> Không có sự khác biệt đáng kể giữa các mẫu xe dựa trên điểm số (p-value = 0.9913 > 0.05)


bai 3

In [12]:
cursor.execute("""
CREATE TABLE flights (
    id integer primary key,
    departure_time integer
)
""")

In [13]:
test_data = [
    (1, 730),
    (2, 1234),
    (3, 45),    
    (4, 2100),  
    (5, 2345),  
    (6, 59)   
]

cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", test_data)
conn.commit()


In [14]:
cursor.execute('''SELECT 
    id,
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS time
FROM flights
               ''')

for i in cursor:
    print(i)


(1, 730, '07:30')
(2, 1234, '12:34')
(3, 45, '00:45')
(4, 2100, '21:00')
(5, 2345, '23:45')
(6, 59, '00:59')


bai 4:

In [15]:
cursor.execute('''CREATE TABLE data_2(
               gia_tri float
               )''')


In [16]:
values = [(14,), (5,), (7,), (8,), (-2,), (111,), (10,), (11,), (13,), (5,), (199,)]
cursor.executemany("INSERT INTO data_2(gia_tri) VALUES (?)", values)
conn.commit()

In [17]:
cursor.execute("""
SELECT gia_tri FROM data_2
ORDER BY gia_tri
""")
vals = [row[0] for row in cursor.fetchall()]
n = len(vals)
median = (vals[n // 2] if n % 2 == 1 else (vals[n // 2 - 1] + vals[n // 2]) / 2)

# 4. Tính |xi - median|
abs_devs = [abs(v - median) for v in vals]
abs_devs.sort()
mad = (abs_devs[n // 2] if n % 2 == 1 else (abs_devs[n // 2 - 1] + abs_devs[n // 2]) / 2)

# ngoại lệ (vượt 1.5 × MAD)
threshold = 1.5 * mad
outliers = [v for v in vals if abs(v - median) > threshold]

print(f"Median = {median}")
print(f"MAD = {mad}")
print(f"Ngưỡng ngoại lệ (1.5×MAD): ±{threshold}")
print("Giá trị ngoại lệ:", outliers)

Median = 10.0
MAD = 4.0
Ngưỡng ngoại lệ (1.5×MAD): ±6.0
Giá trị ngoại lệ: [-2.0, 111.0, 199.0]


bai 5


In [18]:
#  tao bang
cursor.execute("""
CREATE TABLE Patient (
    id integer primary key,
    last_name TEXT,
    height float,
    weight float
)
""")

In [19]:
#  chen du lieu
patients = [
    (1, 'hoang',  170,49),
    (2, 'van',  181,60),   
    (3, 'dang',  155,51),
    (4, 'hoang',  170,49)
]

cursor.executemany("INSERT INTO Patient VALUES (?, ?, ?, ?)", patients)
conn.commit()


In [20]:
cursor.execute("""
SELECT 
    p1.last_name AS person1, 
    p2.last_name AS person2,
    p1.weight AS weight1, 
    p2.weight AS weight2,
    CASE 
        WHEN p1.last_name = p2.last_name AND p1.weight = p2.weight 
        THEN 'cùng người' 
        ELSE 'khác người' 
    END AS match_status
FROM Patient p1, Patient p2
WHERE p1.id < p2.id;
""")
for i in cursor:
    print(i)

('hoang', 'van', 49.0, 60.0, 'khác người')
('hoang', 'dang', 49.0, 51.0, 'khác người')
('hoang', 'hoang', 49.0, 49.0, 'cùng người')
('van', 'dang', 60.0, 51.0, 'khác người')
('van', 'hoang', 60.0, 49.0, 'khác người')
('dang', 'hoang', 51.0, 49.0, 'khác người')
